In [1]:
import pandas
import re

In [2]:
df = pandas.read_csv('redoing_cs_data.csv')
df.head()

,sent_type,university,sent_id,aligned_to,original_sentence,eng_translation,zh_translation,cs_word_id,first_cs_word_form,first_cs_word_translation,...,surprisal_values,average_surprisal,bilingual_corpus_frequency_negative_log_first_cs_word,bilingual_corpus_frequency_negative_log_first_cs_word_trans,length_first_cs_word_form,dependency_distance,if_it_is_root,if_previous_word_is_punctuation,surprisal_of_previous_word,surprisal_first_cs_word_trans
0,code-switch,CMU,2,CMU_296,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,Rich neighborhood is very safe (it belongs to ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,...,6.100998 1.320391 0.7630151 4.4722 3.20577 2.7...,3.144718,11.319098,10.402807,4,1,0,NaN,NaN,6.100998
1,code-switch,CMU,4,CMU_3006,lease 从 八月 开始 ， 具体 日期 可以 商量 。,Lease will begin in August and the date is neg...,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,...,6.527233 3.646975 4.193077 1.173965 0.5694761 ...,3.155464,7.348806,10.066335,5,8,0,NaN,NaN,6.527233
2,code-switch,CMU,5,CMU_70,house 一共 有 两 层 + 地下室 。,House has two floors plus basement.,房子 一共 有 两 层 + 地下室 。,1,house,房子,...,5.512905 5.228868 0.5955775 1.38895 2.136289 4...,3.268832,6.694125,7.084991,5,2,0,NaN,NaN,5.512905
3,code-switch,CMU,6,CMU_6777,一共 有 两 个 可以 洗澡 的 bathroom 。,There are two bathroom for bathing.,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,...,4.509301 0.2903937 1.38895 0.4321387 3.465795 ...,2.642274,8.611048,9.709660,8,6,0,0.0,0.895995,5.210801
4,code-switch,CMU,9,CMU_4373,整个 house 家具 齐全 。,The whole house is fully furnished.,整个 房子 家具 齐全 。,2,house,房子,...,3.732348 4.488641 5.760885 6.166569 0.8237057,4.194430,6.694125,7.084991,5,1,0,0.0,3.732348,4.488641


In [3]:
sents = df['original_sentence']

In [4]:
len(sents)

1476

In [5]:
sents[0]

'rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还有 配套 的 各 种 球场 和 社区 设施 ， 环境 很 好 很 安静 。'

In [6]:
type(sents[0])

str

In [7]:
without_punct = []
for sent in sents:
    stripped = re.sub(r'[^\w\s]', '', sent)
    without_punct.append(stripped)

In [8]:
without_punct[0]

'rich neighborhood  非常 安全  这里 属于 富人 区   旁边 就 是 公园  附近 还有 配套 的 各 种 球场 和 社区 设施  环境 很 好 很 安静 '

In [9]:
split_sent = without_punct[0].split()

In [10]:
split_sent[2].isascii()

False

In [11]:
def process(sent):
    before_cs = []
    cs_back = []
    after_cs = []

    bools = []
    split = sent.split()
    for char in split:
        bools.append(char.isascii() and char.isalpha())

    if bools[0]: #sentence starts with a code-switch
        #print('here in if')
        first_false = bools.index(False)
        cs_back.append(split[first_false])
        after_cs.extend(split[first_false+1:])

    elif bools[-1]: #sentence ends with a code-switch e.g. F, F, F, T
        #print('here in elif')
        num_true = bools.count(True)
        if False not in bools[-num_true:]: #all the Trues are at the end together i.e. only ONE code-switch at the end
            reversed_bools = bools[::-1] #e.g. T, F, F, F
            last_false = reversed_bools.index(False) #e.g. 1
            before_cs.extend(split[:-(last_false)])
        else:
            for pos in range(len(bools)):
                if bools[pos]:
                    before_cs.extend(split[:pos])
                    new_bools = bools[pos:]
                    new_split = split[pos:]
                    break

            false_index = new_bools.index(False)
            cs_back.append(new_split[false_index])
            after_cs.extend(new_split[false_index+1:])  

    else: #code-switch somewhere in the middle of sentence
        #print('here in else')
        for pos in range(len(bools)):
            #e.g. [F, F, F, F, F, T, T, T, F, F]
            if bools[pos]:
                before_cs.extend(split[:pos])
                new_bools = bools[pos:]
                new_split = split[pos:]
                #e.g. [T, T, T, F, F]
                break

        false_index = new_bools.index(False)
        cs_back.append(new_split[false_index])
        after_cs.extend(new_split[false_index+1:])     

    return(" ".join(before_cs), " ".join(cs_back), " ".join(after_cs))

In [12]:
process(without_punct[37])

('想 找 12 间 带 家具 的', '或者', 'apartment room')

In [13]:
without_punct[37]

'想 找 12 间 带 家具 的 house room 或者 apartment room '

In [14]:
before_cs_full = []
cs_back_full = []
after_cs_full = []

for sent in without_punct:
    before_cs_full.append(process(sent)[0])
    cs_back_full.append(process(sent)[1])
    after_cs_full.append(process(sent)[2])
    
print(before_cs_full[566])
print(cs_back_full[566])
print(after_cs_full[566])
    

有 被子
特性
暖和 又 轻便


In [16]:
before_cs_full[566].split()[-1]

'被子'

In [135]:
#print(after_cs_full)

In [17]:
textfile = open("code_switched_sentences_word_before_cs.txt", "w")
for elt in before_cs_full:
    if len(elt)>0:
        split = elt.split()
        textfile.write(split[-1] + "\n")
textfile.close()

In [15]:
textfile = open("code_switched_sentences_before_cs.txt", "w")
for elt in before_cs_full:
    if len(elt)>0:
        textfile.write(elt + "\n")
textfile. close()

In [16]:
textfile = open("code_switched_sentences_cs_back.txt", "w")
for elt in cs_back_full:
    if len(elt)>0:
        textfile.write(elt + "\n")
textfile. close()

In [17]:
textfile = open("code_switched_sentences_after_cs.txt", "w")
for elt in after_cs_full:
    if len(elt)>0:
        textfile.write(elt + "\n")
textfile. close()